# Predicting Diabetes in Pima Indian Women Using Logistic Regression

By Inder Khera, Javier Martinez, Jenny Zhang & Jessica Kuo (alphabetically ordered), 2024/11/23

In [1]:
import numpy as np
import pandas as pd
import altair as alt
import altair_ally as aly
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import (
    RandomizedSearchCV,
    cross_validate,
    cross_val_score,
    train_test_split,
)
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression

from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    ClassImbalance, 
    PercentOfNulls,
    OutlierSampleDetection,
    DataDuplicates,
    MixedDataTypes,
    FeatureLabelCorrelation, 
    FeatureFeatureCorrelation
)
from deepchecks.tabular.checks.data_integrity import PercentOfNulls
import warnings

# Summary


# Introduction

Diabetes is a serious chronic disease characterized by high levels of glucose in the blood, caused by either insufficient insulin production by the pancreas or the body’s inability to effectively use insulin. It has become a significant global health issue, with its prevalence nearly doubling since 1980, and in 2022, 14% of adults aged 18 and older were diagnosed with diabetes, doubling from 7% in 1990 (World Health Organization). Diabetes can lead to severe complications, including blindness, kidney failure, heart attacks, strokes, and lower limb amputations. Early detection enables timely interventions, reduces complications, lowers healthcare costs, and improves quality of life and long-term outcomes (Marshall & Flyvbjerg, 2006).

Artificial intelligence (AI) leverages computer systems and big data to simulate intelligent behavior with minimal human intervention, and within it, machine learning (ML) is a subset of AI methodologies. Machine learning has increasingly been applied in various areas of disease detection and prevention in the healthcare field (Bini, 2018). Numerous machine learning techniques have been deployed to develop more efficient and effective methods for diagnosing chronic diseases (Battineni, Chinatalapudi, & Amenta, 2020). Utilizing machine learning methods in diabetes research is a critical strategy for harnessing large volumes of diabetes-related data to extract valuable insights (Agarwal & Vadiwala, 2022). Therefore, The goal of this report is to leverage a supervised machine learning model, logistic regression (LR), to evaluate its predictive performance in diagnosing diabetes using a real-world dataset focused specifically on Pima Indian women aged 21 and older.

# Methods

### Data

The dataset that was used for the analysis of this project was created by Jack W Smith, JE Everhart, WC Dickson, WC Knowler, RS Johannes and sourced from the National Librabry of Medicine database from the National Institues of Health. Access to their respective analysis can be found [here](https://pmc.ncbi.nlm.nih.gov/articles/PMC2245318/) and access to the dataset can be found via [kaggle](https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data) (Dua & Graff, 2017). The primary objective of the dataset is to enable diagnostic prediction of whether a patient has diabetes based on specific diagnostic measurements. To ensure consistency and relevance, several constraints were applied to the selection of data instances. Specifically, the dataset includes only female patients who are at least 21 years old and of Pima Indian heritage.

Each row/obersvation from the dataset is an individual that identifies to be a part of the Pima (also known as The Akimel O'odham) Indeginous group, located mainly in the Central and Southern regions of the United States. Each observation recorded has summary statistics regarding features that include the Age, BMI, Blood Pressure, Number of Pregnancies, as well as The Diabetes Pedigree Function (which is a score that gives an idea about how much correlation is between person with diabetes and their family history). The dataset offers comprehensive feastures for machine learning analysis.

### Analysis

Logistic Regression was employed to develop a classification model for predicting whether the patient is diabetic or not (as indicated in the `outcome` column of the dataset). All variables from the original dataset were used to train the model. The data was split into 70% for the training set and 30% for the test set. Hyperparameter tuning was performed using `RandomizedSearchCV`, with the accuracy score serving as the classification metric. All variables were standardized just before model fitting. The analysis was conducted using the Python programming language (Van Rossum and Drake, 2009) and several Python packages: numpy (Harris et al., 2020), Pandas (McKinney, 2010), altair (VanderPlas, 2018), altair_ally (Ostblom, 2021) and scikit-learn (Pedregosa et al., 2011). The code used for this analysis and report is available at: https://github.com/UBC-MDS/diabetes_predictor_py

In [2]:
df = pd.read_csv('../data/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


The `shape` attribute shows us the number of observations and the number of features in the dataset

In [3]:
df.shape

(768, 9)

The `info()` method shows that the data set does not have any features with missing values. It further shows that all features are numeric as well.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


Using the `train_test_split()` function we will split our data set with 70% going to train the model and 30% going towards testing the model.

In [5]:
train_df, test_df = train_test_split(df,
                                     train_size = 0.7, 
                                     random_state=123)

The `describe()` shows us the summary statistics of each of our features as well as our target value. We can see the mean as well as the spread (standard deviation). Using this information and the visualization tools we will see next we can determine how skewed each of our features are for their respective values.

In [6]:
census_summary = train_df.describe()
census_summary

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000,537.000000
mean,3.810056,120.337058,69.247672,20.702048,81.960894,32.091806,0.463048,33.344507,0.335196
std,3.318488,31.744549,18.874886,15.677625,116.475625,7.562070,0.331082,11.851165,0.472499
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.200000,0.237000,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,37.000000,32.000000,0.366000,29.000000,0.000000
75%,6.000000,140.000000,80.000000,33.000000,128.000000,36.600000,0.600000,41.000000,1.000000
max,15.000000,199.000000,122.000000,63.000000,744.000000,59.400000,2.420000,81.000000,1.000000


In [7]:
features = census_summary.columns.tolist()
features

['Pregnancies',
 'Glucose',
 'BloodPressure',
 'SkinThickness',
 'Insulin',
 'BMI',
 'DiabetesPedigreeFunction',
 'Age',
 'Outcome']

In [8]:
feature_histograms = alt.Chart(train_df).transform_calculate(
).mark_bar(opacity=0.5).encode( x = alt.X(alt.repeat()).type(
'quantitative').bin(maxbins=30), y= alt.Y('count()').stack(False),
                               color = 'Outcome:N'
).properties( height=250,
width=250 ).repeat(
features, columns=2
)

feature_histograms 

alt.RepeatChart(...)

Figure 1. Comparison of the empirical distributions of training data predictors between those non-diabetic and diabetic

Figure 1 above shows us the respective distribution of each of the features. We have categorized the results to show the how distribution of each feature is when the Outcome is 0 (Non-Diabetic) and when the Outcome is 1 (Diabetic). This helps give us an indication on certain relationships between the features and the target.

For the Glucose levels, we see for the Non-Diabetic class that glucose levels are a somewhat normal distribution; but for the Diabetic class, the glucose levels lean heavily towards the middle to higher range. BMI for the Diabetic class looks like a normal distribution, but it also skews slighty to higher values. But for the Non-Diabetic class interestingly the BMI distribution seems more bimodal.

For the distribution of Age we see that Ages 20 to 32 are dominated by Non-Diabetics, but after the age of 32 we see that the count levels are close between the Diabetic and Non-Diabetic classes, where for some bins the Diabetic class even overtakes the Non-Diabetic even with a lower total count of observations in the data set. The Non-Diabetic class in the Age Distribution leans more towards lower ages meanwhile the Diabetic class' distribution is somewhat consistent across its age range.

For Pregnancies the lower range of pregnancies is dominated by the Non-Diabetic class, meanwhile for higher range of pregnancies the Diabetic class has more observations.

For Skin Thickness both the Diabetic and Non-Diabetic class are close to a normal distribution but the Non-Diabetic distribution skews slighty towards lower values and the Diabetic class skews more towards higher values.

In [9]:
# validate training data for class imbalance for target variable 
# Do these on training data as part of EDA! 
train_df_ds = Dataset(train_df, label = 'Outcome', cat_features=[])

check_lab_cls_imb = ClassImbalance().add_condition_class_ratio_less_than(0.5)
check_lab_cls_imb_result = check_lab_cls_imb.run(dataset = train_df_ds)

if check_lab_cls_imb_result.passed_conditions():
    raise ValueError("Class imbalance exceeds the maximum acceptable threshold.")

In [10]:
# validate training data for percent of nulls
check_pct_nulls = PercentOfNulls().add_condition_percent_of_nulls_not_greater_than(0.05)
check_pct_nulls_result = check_pct_nulls.run(dataset = train_df_ds)

if not check_pct_nulls_result.passed_conditions():
    raise ValueError("Percent of nulls exceeds the maximum acceptable threshold for at least one column.")

In [11]:
# validate training data for percent of outlier samples using loOP algo
check_out_sample = (
    OutlierSampleDetection(nearest_neighbors_percent = 0.01, extent_parameter = 3)
    .add_condition_outlier_ratio_less_or_equal(max_outliers_ratio = 0.001, outlier_score_threshold = 0.9)
)
check_out_sample_result = check_out_sample.run(dataset = train_df_ds)

if not check_out_sample_result.passed_conditions():
    raise ValueError("Number of outlier samples exceeds the maximum acceptable threshold.")

In [12]:
# validate training data for data duplicates
# set duplicate condition to 0 as would not expect any two patient with the exact same situation
check_data_dup = DataDuplicates().add_condition_ratio_less_or_equal(0)
check_data_dup_result = check_data_dup.run(dataset = train_df_ds)

if not check_data_dup_result.passed_conditions():
    raise ValueError("Data duplicates exceed the maximum acceptable threshold.")

In [13]:
# validate training data for mixed data types across all columns
check_mix_dtype = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.2))
check_mix_dtype_result = check_mix_dtype.run(dataset = train_df_ds)

if not check_mix_dtype_result.passed_conditions():
    # raise a warning instead of an error in this case
    warnings.warn("Percentage of rare data type in dangerous zone for at least one column")

In [14]:
aly.corr(train_df)

alt.ConcatChart(...)

Figure 2: Pearson and Spearman correlations across all features

Figure 2 shows the correlation between all of the respective features. The main reasoning to analyze thi is to see if there is any multicollinearity between any of the features which is problamatic when conducting a Logistic Regression. We see that highest level of correlation is between Age and Pregnancies (0.53 by Pearson, and 0.59 via Spearman). Since this is below the threshold of 0.7 we can conclude that all feature coefficients are suitable and will not cause any multicollinearity in our model. 

In [15]:
aly.pair(train_df[features].sample(300), color='Outcome:N')

alt.VConcatChart(...)

Figure 3: Pairwise scatterplots between each of features in dataset to visualize relationship

Figure 3 above gives us a visualization between the relationship between each of our features. We see for the most part that the features do not show and trends. The two features that do show somewhat of a relationship visually is Skin thickness and BMI. This would makes sense as the higher the body mass the higher the thickness of skin would be for the most part. 

Looking back at our previous at the correlation graph from before we see that Skin Thickness and BMI have a Pearson correlation of 0.41, meaning they do not cause multicollinearity in our model. 

In [16]:
# validate training data for anomalous correlations between target/response variable 
# and features/explanatory variables, 
# as well as anomalous correlations between features/explanatory variables

check_feat_lab_corr = FeatureLabelCorrelation().add_condition_feature_pps_less_than(0.7)
check_feat_lab_corr_result = check_feat_lab_corr.run(dataset = train_df_ds)

check_feat_feat_corr = FeatureFeatureCorrelation().add_condition_max_number_of_pairs_above_threshold(threshold = 0.7, n_pairs = 0)
check_feat_feat_corr_result = check_feat_feat_corr.run(dataset = train_df_ds)

if not check_feat_lab_corr_result.passed_conditions():
    raise ValueError("Feature-Label correlation exceeds the maximum acceptable threshold.")

if not check_feat_feat_corr_result.passed_conditions():
    raise ValueError("Feature-feature correlation exceeds the maximum acceptable threshold.")

Here we further split our data set into our X and y for both the training and test

In [17]:
X_train = train_df.drop(columns = ['Outcome'])
y_train = train_df['Outcome']
X_test = test_df.drop(columns = ['Outcome'])
y_test = test_df['Outcome']

We have created a Dummy Classifier to act as our base line for conductin our analysis.
The Dummy Baseline gives us a score of around 0.6648

In [18]:
dummy_clf = DummyClassifier()
mean_cv_score = cross_val_score(dummy_clf, 
                                X_train,
                                y_train).mean()
mean_cv_score

0.6647975077881619

We will be using a Logistic Regression model to do our classification. Since our features have outliers it would be best to use a StandardScaler() to normalize the feature values before fitting the model to them.

In [19]:
log_pipe=make_pipeline(StandardScaler(), LogisticRegression(max_iter=2000,
random_state=123))

We optimze the hyperparamter 'C' for our logistic regression using a random search

In [20]:
np.random.seed(123)
param_dist = {
    "logisticregression__C": [10**i for i in range(-5,15)] 
}

In [21]:
random_search = RandomizedSearchCV(log_pipe,param_dist,
                                   n_iter=20,
                                   n_jobs=-1,
                                   return_train_score=True,
                                   random_state=123)

random_search.fit(X_train,y_train)

RandomizedSearchCV(estimator=Pipeline(steps=[('standardscaler',
                                              StandardScaler()),
                                             ('logisticregression',
                                              LogisticRegression(max_iter=2000,
                                                                 random_state=123))]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'logisticregression__C': [1e-05, 0.0001,
                                                                  0.001, 0.01,
                                                                  0.1, 1, 10,
                                                                  100, 1000,
                                                                  10000, 100000,
                                                                  1000000,
                                                                  10000000,
                                                                  100000000,
                                                                  1000000000,
                                                                  10000000000,
                                                                  100000000000,
                                                                  1000000000000,
                                                                  10000000000000,
                                                                  100000000000000]},
                   random_state=123, return_train_score=True)

We find out best parameter value for our hyperparameter `C` that we will use in our model

In [22]:
best_params = random_search.best_params_ 
best_params

{'logisticregression__C': 10}

In [23]:
pd.DataFrame(random_search.cv_results_).sort_values(
    "rank_test_score").head(3)[["mean_test_score",
                                "mean_train_score"]]

,mean_test_score,mean_train_score
9,0.761717,0.771416
17,0.761717,0.771416
16,0.761717,0.771416


Having determined the best Logistic Regression model for our analysis, we further explore feature importance with coefficients.

In [24]:
# Best model from the search
best_model = random_search.best_estimator_

# Retrieve the coefficients and feature names
coefficients = best_model.named_steps['logisticregression'].coef_.flatten()
features = X_train.columns  

# Create a DataFrame to display the feature names and corresponding coefficients
coeff_df = pd.DataFrame({
    'Features': features,
    'Coefficients': coefficients
})

# Sort by 'Coefficients' in descending order to see the most important features first
coeff_df_sorted = coeff_df.sort_values(by = 'Coefficients', ascending = False)

# Create a heatmap for the coefficients (we will visualize them as a single column)
coeff_df_sorted.style.format(
    precision = 2
).background_gradient(
    axis = None,
    cmap = 'RdBu_r',
    low = 0
)

,Features,Coefficients
1,Glucose,1.08
5,BMI,0.72
0,Pregnancies,0.39
6,DiabetesPedigreeFunction,0.29
7,Age,0.12
3,SkinThickness,0.04
4,Insulin,-0.20
2,BloodPressure,-0.21


Table 1: Logistic regression feature importance measured by coefficients

Based on the heatmap and table 1 above, the feature importance coefficients for the logistic regression model predicting diabetes reveal that `Glucose` (1.08) is the strongest positive influence, followed by `BMI` (0.72) and `Pregnancies` (0.39). The negative influences `BloodPressure` (-0.21) and `Insulin` (-0.20) along with the remaining postive features `DiabetesPedigreeFunction` (0.29), `Age` (0.12), and `SkinThickness` (0.04), have a moderate to weak impact on the prediction, with their effects being less pronounced. 

We then perform best Logistic Regression model from hyperparameter search on the test set.

In [25]:
# Make predictions using the best model
y_pred = best_model.predict(X_test)

In addition, to enhance the model's practical use in a clinical setting, we are providing and reporting probability estimates for the predictions of diabetes. Offering probability estimates would allow clinicians to gauge the model's confidence in its predictions. This would give clinicians the opportunity to conduct additional diagnostic tests if the predicted probability for the outcome (i.e. diagnosis of prediction) is not sufficiently high.

In [26]:
y_pred_prob = best_model.predict_proba(X_test)
pred_bool = (y_test == y_pred)
pred_results_1 = np.vstack([y_test, y_pred, pred_bool, y_pred_prob[:, 1]])
pred_results_1_df = pd.DataFrame(pred_results_1.T, 
                                 columns = ['y_test', 'y_pred', 'pred_bool', 'y_pred_prob_1'])
pred_results_1_df['pred_bool'] = pred_results_1_df['pred_bool'] == 1
pred_results_1_df.head()

,y_test,y_pred,pred_bool,y_pred_prob_1
0,1.0,1.0,True,0.860446
1,0.0,0.0,True,0.353960
2,0.0,1.0,False,0.652235
3,0.0,0.0,True,0.053024
4,0.0,0.0,True,0.073861


Our prediction model performed decent on test data, with a final overall accuracy of 0.80. In addition, looking through the prediction results dataframe, there are a total of 46 mistakes. Of which, 31 mistakes were predicting diabetic as non-diabetic (false negatives) and 15 mistakes were made predicting diabetic as non-diabetic (false positives). Considering implementation in clinic, there is room for improvement in the algorithm as false negatives are more harmful than false positives, and we should aim to lower false positives even further.

In [27]:
# Compute accuracy
accuracy = best_model.score(X_test, y_test)

pd.DataFrame({'accuracy': [accuracy]})

,accuracy
0,0.800866


In [28]:
# Calculate the number of correct predictions and misclassifications
value_counts = pred_results_1_df['pred_bool'].value_counts()

pd.DataFrame({
    'correct predictions': [value_counts.get(True, 0)], 
    'misclassifications': [value_counts.get(False, 0)]
})

,correct predictions,misclassifications
0,185,46


In [29]:
# Calculate the number of false positives (FPs) and false negatives (FNs)
fp = len(pred_results_1_df[(pred_results_1_df['y_test'] == 0) & (pred_results_1_df['y_pred'] == 1)])
fn = len(pred_results_1_df[(pred_results_1_df['y_test'] == 1) & (pred_results_1_df['y_pred'] == 0)])

pd.DataFrame({
    'false positives': [fp], 
    'false negatives': [fn]
})

,false positives,false negatives
0,15,31


Moreover, visualizing prediction probabilities alongside the prediction accuracy for each test sample provides a clearer understanding of the model's performance. This approach allows us to easily assess how well the model predicts, while also highlighting patients who were misdiagnosed. Particularly, it helps us focus on false negatives, as the consequences of these errors are more critical in a clinical context.

In [30]:
alt.Chart(pred_results_1_df, title = 'Test Set Prediction Accuracy').mark_tick().encode(
    x = alt.X('y_pred_prob_1').title('Positive Class Prediction Prob'),
    y = alt.Y('pred_bool').title('Pred. Accuracy'),
    color = alt.Color('y_test:N').title('Outcome')
)

alt.Chart(...)

Figure 4: Test Set Prediction Accuracy by Prediction Probability

While the performance of this model may be valuable as a screening tool in a clinical context, especially given its improvements over the baseline, there are several opportunities for further enhancement. One potential approach is to closely examine the 46 misclassified observations, comparing them with correctly classified examples from both classes. The objective would be to identify which features may be contributing to the misclassifications and investigate whether feature engineering could help the model improve its predictions on the observations it is currently struggling with. Additionally, we would try seeing whether we can get improved predictions using other classifiers. Other classifiers we might try are 1) random forest because it automatically allows for feature interaction, 2) k-nearest neighbours (k-NN) which usually provides easily interpretable and decent predictions, and 3) support vector classifier (SVC) as it allows for non-linear prediction using the rbf kernel. Finally, there runs the possibility that the features offered from this dataset alone are not sufficient to predict with high accuracy. In this case, conducting additional conversations with data collectors for additional useable information or explore additional datasets that can be joined so our set of features can be expanded for more complicated analysis might be beneficial. 

At last, we recognize the limitation with this dataset, as it focuses solely on Pima Indian women aged 21 and older, which limits its generalizability to other populations. To improve the analysis, it would be valuable to combine this data with other datasets representing different age groups, genders, and ethnicities, enabling more comprehensive insights and broader applicability of the findings.

# References

Agarwal, N., & Vadiwala, R. (2022). Machine Learning and Data Mining Methods in Diabetes Research. Asian Journal of Organic & Medicinal Chemistry.
 
Battineni, G., Sagaro, G. G., Chinatalapudi, N., & Amenta, F. (2020). Applications of machine learning predictive models in the chronic disease diagnosis. Journal of personalized medicine, 10(2), 21.
 
Bini, S. A. (2018). Artificial intelligence, machine learning, deep learning, and cognitive computing: what do these terms mean and how will they impact health care?. The Journal of arthroplasty, 33(8), 2358-2361.
 
Dua, D., & Graff, C. (2017). Pima Indians Diabetes Database. UCI Machine Learning Repository. Retrieved from https://www.kaggle.com/datasets/uciml/pima-indians-diabetes-database/data
 
Marshall, S. M., & Flyvbjerg, A. (2006). Prevention and early detection of vascular complications of diabetes. Bmj, 333(7566), 475-480.
  
World Health Organization. (n.d.). Diabetes. Retrieved November 22, 2024, from https://www.who.int/news-room/fact-sheets/detail/diabetes